In [55]:
# The code was removed by Watson Studio for sharing.

## Import Your Dataset

In [1]:
# The code was removed by Watson Studio for sharing.

,Unnamed: 0,Months since Last Donation,Number of Donations,Total Volume Donated (c.c.),Months since First Donation,Made Donation in March 2007
0,619,2,50,12500,98,1
1,664,0,13,3250,28,1
2,441,1,16,4000,35,1
3,160,2,20,5000,45,1
4,358,1,24,6000,77,0


In [3]:

body = client_887e8fed66994bce85b726a4b6eee3dd.get_object(Bucket='predictblooddonation-donotdelete-pr-zd9e3njxbqzk3r',Key='test.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_test = pd.read_csv(body)
df_test.head()



,Unnamed: 0,Months since Last Donation,Number of Donations,Total Volume Donated (c.c.),Months since First Donation
0,659,2,12,3000,52
1,276,21,7,1750,38
2,263,4,1,250,4
3,303,11,11,2750,38
4,83,4,12,3000,34


In [19]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

In [5]:
df_train.shape#check the shape of the data

(576, 6)

In [6]:
np.sum(df_train.duplicated())# no duplicates

0

In [7]:
#check the null values
df_train.isnull().sum()

Unnamed: 0                     0
Months since Last Donation     0
Number of Donations            0
Total Volume Donated (c.c.)    0
Months since First Donation    0
Made Donation in March 2007    0
dtype: int64

In [8]:
df_test.isnull().sum()

Unnamed: 0                     0
Months since Last Donation     0
Number of Donations            0
Total Volume Donated (c.c.)    0
Months since First Donation    0
dtype: int64

#### No null values

In [10]:
df_train.dtypes

Unnamed: 0                     int64
Months since Last Donation     int64
Number of Donations            int64
Total Volume Donated (c.c.)    int64
Months since First Donation    int64
Made Donation in March 2007    int64
dtype: object

In [30]:
df_train.corr()

,Unnamed: 0,Months since Last Donation,Number of Donations,Total Volume Donated (c.c.),Months since First Donation,Made Donation in March 2007
Unnamed: 0,1.000000,-0.023217,0.063148,0.063148,0.098370,0.041925
Months since Last Donation,-0.023217,1.000000,-0.159731,-0.159731,0.186899,-0.261234
Number of Donations,0.063148,-0.159731,1.000000,1.000000,0.622116,0.220615
Total Volume Donated (c.c.),0.063148,-0.159731,1.000000,1.000000,0.622116,0.220615
Months since First Donation,0.098370,0.186899,0.622116,0.622116,1.000000,-0.019819
Made Donation in March 2007,0.041925,-0.261234,0.220615,0.220615,-0.019819,1.000000


In [11]:
df_train.columns

Index(['Unnamed: 0', 'Months since Last Donation', 'Number of Donations',
       'Total Volume Donated (c.c.)', 'Months since First Donation',
       'Made Donation in March 2007'],
      dtype='object')

In [12]:
x=df_train[['Months since Last Donation', 'Number of Donations',
       'Total Volume Donated (c.c.)', 'Months since First Donation']].values

In [13]:
y=df_train['Made Donation in March 2007'].values

In [14]:
x=preprocessing.StandardScaler().fit(x).transform(x)

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


#### Split Your Dataset

In [15]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=4)

In [16]:
print('x_train',x_train.shape)
print('y_train',y_train.shape)

x_train (460, 4)
y_train (460,)


* Data was split well

In [22]:
model1=DecisionTreeClassifier(criterion='entropy',max_depth=4)
model2=LogisticRegression(C=0.01,solver='liblinear')
model3=KNeighborsClassifier(n_neighbors=6)

In [20]:
Ks = 10
mean_acc = np.zeros((Ks-1))
std_acc = np.zeros((Ks-1))
ConfustionMx = [];
for n in range(1,Ks):
    
    #Train Model and Predict  
    neigh = KNeighborsClassifier(n_neighbors = n).fit(x_train,y_train)
    yhat=neigh.predict(x_test)
    mean_acc[n-1] = metrics.accuracy_score(y_test, yhat)

    
    std_acc[n-1]=np.std(yhat==y_test)/np.sqrt(yhat.shape[0])

mean_acc

array([ 0.68965517,  0.78448276,  0.78448276,  0.80172414,  0.75862069,
        0.81896552,  0.80172414,  0.78448276,  0.81896552])

In [21]:
print( "The best accuracy was with", mean_acc.max(), "with k=", mean_acc.argmax()+1)

The best accuracy was with 0.818965517241 with k= 6


### Train your models

In [24]:
model1.fit(x_train,y_train)
model2.fit(x_train,y_train)
model3.fit(x_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=6, p=2,
           weights='uniform')

### Predict your outcome

In [25]:
y_hat1=model1.predict(x_test)
y_hat2=model2.predict(x_test)
y_hat3=model3.predict(x_test)

In [26]:
print('predicted',y_hat1[0:5])
print('actual',y_test[0:5])

predicted [0 0 1 0 0]
actual [0 0 1 0 0]


In [39]:
y_hat1prob=model1.predict_proba(x_test)
y_hat2prob=model2.predict_proba(x_test)
y_hat3prob=model3.predict_proba(x_test)

### Evaluation

In [27]:
from sklearn.metrics import jaccard_similarity_score
jaccard_similarity_score(y_test,y_hat1)

0.81896551724137934

In [28]:
from sklearn.metrics import jaccard_similarity_score
jaccard_similarity_score(y_test,y_hat2)

0.81896551724137934

In [29]:
from sklearn.metrics import jaccard_similarity_score
jaccard_similarity_score(y_test,y_hat3)

0.81896551724137934

## Log Loss

1.0254684538134027

In [41]:
log_loss(y_test,y_hat2prob)

0.52291698282628485

In [42]:
log_loss(y_test,y_hat3prob)

2.7214711750141163

### Testing

In [31]:
df_test.head()

,Unnamed: 0,Months since Last Donation,Number of Donations,Total Volume Donated (c.c.),Months since First Donation
0,659,2,12,3000,52
1,276,21,7,1750,38
2,263,4,1,250,4
3,303,11,11,2750,38
4,83,4,12,3000,34


In [32]:
df_test.columns

Index(['Unnamed: 0', 'Months since Last Donation', 'Number of Donations',
       'Total Volume Donated (c.c.)', 'Months since First Donation'],
      dtype='object')

In [33]:
test_x=df_test[['Months since Last Donation', 'Number of Donations',
       'Total Volume Donated (c.c.)', 'Months since First Donation']].values

In [34]:
test_x=preprocessing.StandardScaler().fit(test_x).transform(test_x)

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


### predict

In [35]:
yhat1=model1.predict_proba(test_x)
yhat2=model2.predict_proba(test_x)
yhat3=model3.predict_proba(test_x)

In [36]:
yhat1[0:5]

array([[ 0.84210526,  0.15789474],
       [ 0.        ,  1.        ],
       [ 0.75409836,  0.24590164],
       [ 0.88481675,  0.11518325],
       [ 0.275     ,  0.725     ]])

In [37]:
Decision=pd.DataFrame(yhat1)

In [38]:
Decision.head()

,0,1
0,0.842105,0.157895
1,0.000000,1.000000
2,0.754098,0.245902
3,0.884817,0.115183
4,0.275000,0.725000


In [57]:
Decision.columns=['zero','Made Donation in March 2007']

In [58]:
df_final=pd.concat([df_test,Decision],axis=1)
df_final.head()

,Unnamed: 0,Months since Last Donation,Number of Donations,Total Volume Donated (c.c.),Months since First Donation,zero,Made Donation in March 2007
0,659,2,12,3000,52,0.842105,0.157895
1,276,21,7,1750,38,0.000000,1.000000
2,263,4,1,250,4,0.754098,0.245902
3,303,11,11,2750,38,0.884817,0.115183
4,83,4,12,3000,34,0.275000,0.725000


In [47]:
df_final.columns

Index(['Unnamed: 0', 'Months since Last Donation', 'Number of Donations',
       'Total Volume Donated (c.c.)', 'Months since First Donation', 'zero',
       'one'],
      dtype='object')

In [49]:
df_sub=df_final[['Unnamed: 0','one']]
df_sub.head()

,Unnamed: 0,one
0,659,0.157895
1,276,1.000000
2,263,0.245902
3,303,0.115183
4,83,0.725000


In [59]:
logistic=pd.DataFrame(yhat2)
logistic.columns=['zero','Made Donation in March 2007']
logistic.head()

,zero,Made Donation in March 2007
0,0.524924,0.475076
1,0.710904,0.289096
2,0.616586,0.383414
3,0.597224,0.402776
4,0.525910,0.474090


In [60]:
df_final2=pd.concat([df_test,logistic],axis=1)
df_final2.head()

,Unnamed: 0,Months since Last Donation,Number of Donations,Total Volume Donated (c.c.),Months since First Donation,zero,Made Donation in March 2007
0,659,2,12,3000,52,0.524924,0.475076
1,276,21,7,1750,38,0.710904,0.289096
2,263,4,1,250,4,0.616586,0.383414
3,303,11,11,2750,38,0.597224,0.402776
4,83,4,12,3000,34,0.525910,0.474090


In [61]:
df_sub1=df_final2[['Unnamed: 0','Made Donation in March 2007']]
df_sub1.head()

,Unnamed: 0,Made Donation in March 2007
0,659,0.475076
1,276,0.289096
2,263,0.383414
3,303,0.402776
4,83,0.474090


In [62]:
project.save_data(data=df_sub1.to_csv(index=False),file_name='Logistic_prediction.csv',overwrite=True)

{'asset_id': 'e7dd2eae-25d9-4857-a1c9-c7fcd8863af1',
 'bucket_name': 'predictblooddonation-donotdelete-pr-zd9e3njxbqzk3r',
 'file_name': 'Logistic_prediction.csv',
 'message': 'File saved to project storage.'}